In [1]:
import os

import scanpy as sc
import gdown

In [4]:
DATA_PATH = "~/data/netemo"
DATA_PATH = os.path.expanduser(os.path.expandvars(DATA_PATH))
MODEL_DIR = "models"
RESULTS_PATH = "results"
SCRNA_FNAME = 'kotliarov2020-expressions.h5ad'
SCRNA_LINK = 'https://drive.google.com/uc?id=1wA3VBUnYEW2qHPk9WijNTKjV9KriWe8y',
SCCITE_FNAME = 'kotliarov2020-proteins.h5ad'
SCCITE_LINK = 'https://drive.google.com/uc?id=112mdDX76LZRL33tBLYhfYRRXOUrLUhw-'

data = {}

for data_type, data_fname, data_link in [
            ("rna", SCRNA_FNAME, SCRNA_LINK),
            ("cite", SCCITE_FNAME, SCCITE_LINK),
        ]:
    try:
        data[data_type] = sc.read_h5ad(os.path.join(DATA_PATH,data_fname))
    except OSError:
        gdown.download(
            data_link,
            os.path.join(DATA_PATH,data_fname)
        )
        data[data_type] = sc.read_h5ad(os.path.join(DATA_PATH,data_fname))

In [5]:
aaa = sc.read_h5ad(os.path.expanduser(os.path.expandvars("~/Downloads/protein.h5ad")))

In [15]:
assert np.all(aaa.X == data["cite"].X) or np.allclose(aaa.X, data["cite"].X)
for k in aaa.obs_keys():
    assert np.all(aaa.obs[k] == data["cite"].obs[k]) or np.allclose(aaa.obs[k], data["cite"].obs[k])
for k in aaa.obsm_keys():
    assert np.all(aaa.obsm[k] == data["cite"].obsm[k]) or np.allclose(aaa.obsm[k], data["cite"].obsm[k])
for k in aaa.var_keys():
    assert np.all(aaa.var[k] == data["cite"].var[k]) or np.allclose(aaa.var[k], data["cite"].var[k])
for k in aaa.varm_keys():
    assert np.all(aaa.varm[k] == data["cite"].varm[k]) or np.allclose(aaa.varm[k], data["cite"].varm[k])
for k in aaa.uns_keys():
    match k:
        case "neighbors":
            pass
        case "pca":
            assert np.all(aaa.uns[k]["variance"] == data["cite"].uns[k]["variance"]) or np.allclose(aaa.uns[k]["variance"], data["cite"].uns[k]["variance"])
        case _:
            assert np.all(aaa.uns[k] == data["cite"].uns[k]) or np.allclose(aaa.uns[k], data["cite"].uns[k])


for k in aaa.obsp.keys():
    assert not np.any((aaa.obsp[k] != data["cite"].obsp[k]).nonzero()) or np.allclose(aaa.obsp[k], data["cite"].obsp[k])

for k in aaa.layers.keys():
    assert not np.any((aaa.layers[k] != data["cite"].layers[k]).nonzero()) or np.allclose(aaa.layers[k], data["cite"].layers[k])

In [ ]:
import seaborn as sns
import numpy as np

unique_cell_types = sorted(set(data[data_type].obs["cell_type"]))
cell_type_pal = sns.color_palette("colorblind", n_colors=len(unique_cell_types))
cell_type_lut = dict(zip(unique_cell_types,cell_type_pal))
cell_type_rgb_value = np.array([cell_type_lut[c] for c in data[data_type].obs["cell_type"]])

In [ ]:
import scipy.spatial

# Just checking that the 87 values in the cite dataframe aren't actually the "top 87 genes in the brown module" since the paper mentions "82 surface proteins"
corrs = 1 - scipy.spatial.distance.cdist(data["rna"].X.T.toarray(), data["cite"].X.T, metric="correlation")
plt.hist(corrs.max(axis=0))
pass

In [ ]:
import matplotlib.pyplot as plt

for data_type in [
            "rna",
            "cite",
        ]:
    g = sns.scatterplot(
        x=data[data_type].obsm["X_umap"][:,0],
        y=data[data_type].obsm["X_umap"][:,1],
        hue=data[data_type].obs["cell_type"],
        hue_order=unique_cell_types,
        palette=cell_type_pal,
        s=1,
        )
    sns.move_legend(g, loc="upper left", bbox_to_anchor=(1, 1))
    plt.title(data_type)
    plt.show()

In [ ]:
import networkx as nx

node_limit = data[data_type].X.shape[0] #5000

for data_type in [
            "rna",
            "cite",
        ]:
    g = nx.graph.Graph(data[data_type].obsp["connectivities"])
    g_pos = dict(zip(range(len(data[data_type].obsm["X_umap"])),data[data_type].obsm["X_umap"]))
    
    node_sample = np.random.choice(len(cell_type_rgb_value), size=node_limit, replace=False)

    nx.drawing.draw_networkx(
        g.subgraph(node_sample),
        with_labels=False,
        node_size=1,
        width=0.1,
        edge_color="#00000001",
        node_color=cell_type_rgb_value,#[node_sample],
        alpha=0.1,
        pos=g_pos,
        )
    # make empty plot with correct color and label for each group
    for k, v in cell_type_lut.items():
        plt.scatter([],[], c=[v], label=k)
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.title(data_type)
    plt.show()

In [ ]:
import networkx as nx

node_limit = data[data_type].X.shape[0] #5000
g_pos = {}

for data_type in [
            "rna",
            "cite",
        ]:
    g = nx.graph.Graph(data[data_type].obsp["connectivities"])
    if data_type not in g_pos:
        g_pos[data_type] = nx.spring_layout(g)
    
    node_sample = np.random.choice(len(cell_type_rgb_value), size=node_limit, replace=False)

    nx.drawing.draw_networkx(
        g.subgraph(node_sample),
        with_labels=False,
        node_size=1,
        width=0.1,
        edge_color="#00000001",
        node_color=cell_type_rgb_value,#[node_sample],
        alpha=0.1,
        pos=g_pos[data_type],
        )
    # make empty plot with correct color and label for each group
    for k, v in cell_type_lut.items():
        plt.scatter([],[], c=[v], label=k)
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.title(data_type)
    plt.show()